### Fine-tuning a model with the Trainer API

Transformer提供了一个Trainer类来帮助您微调它在数据集上提供的任何预训练模型。一旦完成了上一节中的所有数据预处理工作，就只剩下几个步骤来定义Trainer了。
最困难的部分可能是准备运行Trainer.train()的环境，因为它在CPU上运行得非常慢。
下面的代码示例假设您已经执行了前一节中的示例。以下是你需要的一个简短的总结:

In [1]:
import torch
import numpy as np
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Found cached dataset glue (C:/Users/Jager/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Jager\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-5137f2fb4737d116.arrow
Loading cached processed dataset at C:\Users\Jager\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-60ba0ce5ca5c6a38.arrow
Loading cached processed dataset at C:\Users\Jager\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-b26904918f30dd5d.arrow


In [3]:
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

#### Training

定义训练器之前的第一步是定义一个TrainingArguments类，该类将包含Trainer用于训练和评估的所有超参数。您必须提供的唯一参数是保存经过训练的模型的目录，以及checkpoints。对于其他所有选项，您可以保留默认值，这对于基本的微调应该非常有效。

In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments("./test-trainer")

第二步是定义我们的模型。和前一章一样，我们将使用AutoModelForSequenceClassification类，带有两个标签:

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

你会注意到，与第2章不同的是，在实例化这个预训练模型之后，你会得到一个警告。这是因为BERT没有对句子对分类进行预训练，所以预训练模型的头部被丢弃，取而代之的是一个适合序列分类的新头部。警告表明一些权重没有被使用(那些对应于丢失的预训练头的权重)，而其他一些权重是随机初始化的(那些针对新头的权重)。它的结论是鼓励你训练模型，这正是我们现在要做的。

一旦我们有了模型，我们可以构造一个Trainer对象：

In [7]:
from transformers import Trainer

trainer = Trainer(model,
                  training_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["validation"],
                  data_collator=data_collator,
                  tokenizer=tokenizer)

注意，当您像这里一样传递tokenizer时，训练器使用的默认数据整理器将是前面定义的DataCollatorWithPadding，
因此您可以跳过此调用中的data_collator=data_collator行。在第2节中向您展示这部分处理仍然很重要!
为了在我们的数据集上微调模型，我们只需要调用train()方法即可:

In [8]:
trainer.train()

C:\ProgramData\anaconda3\envs\dlwpt\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=1377, training_loss=0.29651290018141224, metrics={'train_runtime': 139.008, 'train_samples_per_second': 79.161, 'train_steps_per_second': 9.906, 'total_flos': 406183858377360.0, 'train_loss': 0.29651290018141224, 'epoch': 3.0})

这将开始微调(在GPU上应该需要几分钟)，并每500步报告一次训练损失。然而，它不会告诉您模型的表现有多好(或多差)。这是因为:
- 没有通过将evaluation_strategy设置为“steps”(评估每个eval_steps)或“epoch”(在每个epoch的末尾评估)来告诉Trainer在训练期间进行评估。
- 没有为Trainer提供compute_metrics()函数，以便在上述求值期间计算指标(否则，求值只会打印损失，这不是一个非常直观的数字)。

### Evaluation

让我们看看如何构建一个有用的compute_metrics()函数，并在下次训练时使用它。该函数必须接受一个EvalPrediction对象(它是一个带有predictions和label_ids字段的tuple)，并将返回一个将字符串映射到浮点数的字典(字符串是返回的指标的名称，并且浮点它们的值)。
为了从我们的模型中得到一些预测，我们可以使用Trainer.predict()命令:

In [9]:
predictions = trainer.predict(tokenized_datasets["validation"])
predictions.predictions.shape, predictions.label_ids.shape

((408, 2), (408,))

In [10]:
predictions

PredictionOutput(predictions=array([[-3.8238406 ,  3.513597  ],
       [ 3.064186  , -2.8706763 ],
       [-3.160697  ,  2.9403021 ],
       [-3.7238348 ,  3.5117645 ],
       [ 3.0841463 , -2.8048441 ],
       [-3.7747707 ,  3.5051389 ],
       [-3.6928523 ,  3.3666382 ],
       [-3.5853903 ,  3.3992593 ],
       [-3.8030686 ,  3.510004  ],
       [-3.8218102 ,  3.4889898 ],
       [-3.8460193 ,  3.5379267 ],
       [ 2.9890764 , -2.7416606 ],
       [ 3.1132689 , -2.8708358 ],
       [ 0.4085788 , -0.2915823 ],
       [-3.7459266 ,  3.4022703 ],
       [ 2.1240315 , -2.0164144 ],
       [-3.8094642 ,  3.5345862 ],
       [ 2.1404076 , -2.0221896 ],
       [-3.8114698 ,  3.5280488 ],
       [ 2.838272  , -2.6434863 ],
       [ 3.1005507 , -2.819066  ],
       [-3.388288  ,  3.241027  ],
       [-1.7520449 ,  1.7444922 ],
       [-3.7704413 ,  3.5159264 ],
       [-3.5382295 ,  3.3722386 ],
       [ 1.9723662 , -1.6837708 ],
       [-2.9892032 ,  2.9504232 ],
       [-3.8328378 ,  3.49

predict()方法的输出是另一个元组，包含三个字段:predictions、label_ids和metrics。metrics字段将只包含传递的数据集上的损失，以及一些时间指标(预测所需的时间，总数和平均时间)。将函数compute_metrics传递给Trainer之后，该字段还将包含compute_metrics()返回的指标。

正如您所看到的，predictions是一个形状为408 x 2的二维数组(408是我们使用的数据集中元素的数量)。这些是我们传递给predict()的数据集的每个元素的logits(正如您在前一章中看到的，所有Transformer模型都返回logits)。为了将它们转换为可以与标签进行对比的预测值，我们需要在第二个轴上取最大值的索引:

In [11]:
preds = np.argmax(predictions.predictions, axis=-1)
preds

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,

现在我们可以将这些数据与标签进行比较。要构建compute_metric()函数，我们需要依赖Evaluate库中的指标。
我们可以像加载数据集一样轻松地加载与MRPC数据集相关的指标，这次使用evaluate.load()函数，返回的对象有一个compute()方法，可以使用它来进行度量计算:

In [12]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8382352941176471, 'f1': 0.8885135135135136}

您得到的确切结果可能会有所不同，因为模型头的随机初始化可能会改变它所实现的度量。在这里，我们可以看到我们的模型在验证集上的准确率为85.78%，F1分数为89.97。
这是用于评估GLUE基准的MRPC数据集上的结果的两个度量。BERT论文中的表格报告了基础模型的F1得分为88.9。这是我们目前使用的uncase模型，它解释了更好的结果。
将所有内容放在一起，我们得到了compute_metrics()函数:

In [13]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

请注意，我们创建了一个新的TrainingArguments，其evaluation_strategy设置为“epoch”，并创建了一个新模型——否则，我们只是在继续训练我们已经训练过的模型。
开始新的训练:

In [15]:
trainer.train()

C:\ProgramData\anaconda3\envs\dlwpt\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=1377, training_loss=0.3202050152946921, metrics={'train_runtime': 142.9423, 'train_samples_per_second': 76.982, 'train_steps_per_second': 9.633, 'total_flos': 406183858377360.0, 'train_loss': 0.3202050152946921, 'epoch': 3.0})

这一次，每个epoch结束时，在training loss顶部报告validation loss 和 metrics。
同样，由于模型的随机头部初始化，您达到的准确精度/F1分数可能与我们发现的略有不同，但它应该在相同的范围内。

Trainer将在多个gpu或tpu上开箱即用，并提供许多选项，如混合精度训练(在训练参数中使用fp16 = True)。我们将在第10章讨论它支持的所有内容。
本文结束了对使用Trainer API进行微调的介绍。对于大多数常见的NLP任务，将在第7章给出一个这样做的例子，但是现在让我们看看如何在纯PyTorch中做同样的事情。